In [42]:
import os
import sys
import inspect
from rep_client import RepClient

currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parentdir = os.path.dirname(currentdir)
modeldir = os.path.join(parentdir, "models")
sys.path.insert(0,modeldir) 
import importlib

import tensorflow as tf
import numpy as np


from baseline_constants import MAIN_PARAMS, MODEL_PARAMS

from tensorflow.python import pywrap_tensorflow
from utils.model_utils import read_data
from baseline_constants import ACCURACY_KEY

In [49]:
# All initalization section
Learning_Rate = (0.1, 0.3, 0.4)
SEED = 556667788


model_name = "femnist.cnn"
file_part = model_name.split(".")[1]
dataset = model_name.split(".")[0]

# create the mnist model

ckpt_path = "./" + file_part + ".ckpt"
model_params = MODEL_PARAMS[model_name]
base_dir = "/scratch/mlhead-init"

print('############################## %s ##############################' % model_name)
mod = importlib.import_module(model_name)
ClientModel = getattr(mod, 'ClientModel')

tf.reset_default_graph()
client_model = ClientModel(SEED, *model_params)

# load model exported weights
def restore_ckpt(variable, ckpt_path):
    reader = pywrap_tensorflow.NewCheckpointReader( ckpt_path )
    var =  reader.get_tensor( variable ) 
    return var.astype(np.float64) 

def create_clients(users, groups, train_data, test_data, model):
    if len(groups) == 0:
        groups = [[] for _ in users]
    clients = [RepClient(u, g, train_data[u], test_data[u], model) for u, g in zip(users, groups)]
    return clients

def test(model, data):
    """
    Tests the current model on the given data.

    Args:
        data: dict of the form {'x': [list], 'y': [list]}
    Return:
        dict of metrics that will be recorded by the simulation.
    """
    x_vecs = model.process_x(data['x'])
    labels = model.process_y(data['y'])
    with model.graph.as_default():
        tot_acc, loss, pred = model.sess.run(
            [model.eval_metric_ops, model.loss, model.pred_ops],
            feed_dict={model.features: x_vecs, model.labels: labels}
        )
    acc = float(tot_acc) / x_vecs.shape[0]
    metric = {ACCURACY_KEY: acc, 'loss': loss}
    eval_items = {"labels": labels, "pred":pred}
    return metric, eval_items 


from tensorflow.python.tools.inspect_checkpoint import print_tensors_in_checkpoint_file
#print_tensors_in_checkpoint_file(file_name=ckpt_path, tensor_name='v0', all_tensors=True)

with client_model.graph.as_default():
    all_vars = tf.trainable_variables()
    params = [restore_ckpt(v.op.name, ckpt_path) for v in all_vars]
    client_model.set_params(params)

# test the mnist model 
train_data_dir = os.path.join(base_dir, 'data', dataset, 'data', 'train')
test_data_dir = os.path.join(base_dir, 'data', dataset, 'data', 'test')
users, groups, train_data, test_data = read_data(train_data_dir, test_data_dir)
clients = create_clients(users, groups, train_data, test_data, client_model)


# show precision recall f1
# https://scikit-learn.org/stable/modules/generated/sklearn.metrics.classification_report.html

############################## femnist.cnn ##############################


In [53]:
from sklearn.metrics import classification_report, f1_score, precision_score, accuracy_score
from utils.language_utils import letter_to_vec, word_to_indices
import csv

classes = 62
target_names = ["class {}".format(i) for i in range(classes)]

def process_y(raw_y_batch):
    if dataset == "shakespeare":
        y_batch = [np.argmax(letter_to_vec(c))  for c in raw_y_batch]
        return y_batch
    else:
        return raw_y_batch

metrics = {}
tot_score = 0.
tot_prec = 0.
total_samples = 0
acc_list = [0] * len(clients)
sample_list = [0] * len(clients)
index = 0
for cli in clients[:]:
    c_metrics, eval_items = test(client_model, test_data[cli.id])
    ys = process_y(test_data[cli.id]['y']) 
    #ys = test_data[cli.id]['y']
    print(c_metrics)
    total_samples += len(ys)
    metrics[cli.id] = c_metrics
    tot_prec  += precision_score(ys, eval_items["pred"], average='micro')
    tot_score += f1_score(ys, eval_items["pred"], average='macro')  
    acc_list[index] = accuracy_score(ys, eval_items["pred"])
    sample_list[index] = len(ys)
    index += 1
    
sample_list = [s / total_samples for s in sample_list]
print("Micro accuracy: ", np.average(acc_list, weights=None))
print("Micro accuracy: ", np.average(acc_list, weights=sample_list))
print("Avg Precision: ", tot_prec / len(clients))
print("Avg micro f1: ", tot_score / len(clients))

    
# for cli in clients[:]:
#         with open("femnist-saved.tsv", 'a+') as tsvfile:
#             c_metrics, eval_items = test(client_model, test_data[cli.id])
#             ys = test_data[cli.id]['y']
#             print("{}\t{}\t{}".format(cli.id, ys, eval_items["pred"]))
#             writer = csv.writer(tsvfile, delimiter='\t', lineterminator='\n')
#             for i,j in zip(ys, eval_items["pred"]):
#                 writer.writerow([cli.id, i, j])              



{'accuracy': 0.7096774193548387, 'loss': 1.8967067}
{'accuracy': 0.7307692307692307, 'loss': 2.412104}
{'accuracy': 0.84375, 'loss': 0.73260874}
{'accuracy': 0.7560975609756098, 'loss': 0.82100594}
{'accuracy': 0.875, 'loss': 0.71608114}
{'accuracy': 0.6666666666666666, 'loss': 1.1963109}
{'accuracy': 0.8571428571428571, 'loss': 0.57580906}
{'accuracy': 0.7692307692307693, 'loss': 1.1877186}
{'accuracy': 0.6785714285714286, 'loss': 2.5988243}
{'accuracy': 0.7, 'loss': 1.3468858}
{'accuracy': 0.7317073170731707, 'loss': 1.0607078}
{'accuracy': 0.8461538461538461, 'loss': 0.9686256}
{'accuracy': 0.5, 'loss': 2.2558136}
{'accuracy': 0.6666666666666666, 'loss': 3.075478}
{'accuracy': 0.9090909090909091, 'loss': 0.31169286}
{'accuracy': 0.7741935483870968, 'loss': 1.2483513}
{'accuracy': 0.75, 'loss': 2.1453526}
{'accuracy': 0.7727272727272727, 'loss': 2.4446614}
{'accuracy': 0.5454545454545454, 'loss': 3.051976}
{'accuracy': 0.7368421052631579, 'loss': 0.7526577}
{'accuracy': 0.76470588235

In [ ]:
# minist params
#python3 mlhead.py -dataset femnist -model cnn --num-rounds 250  -lr 0.003 \
#  --clients-per-round 2 --num_clusters "${1}" --num-epochs 10  \
#| tee "outdir/femnist-k${1}.out"

# celeba params
#   python3 mlhead.py -dataset celeba -model cnn -t medium -lr 0.03 \
#  --clients-per-round 20--num_clusters "${1}"  --num-epochs 5 \
# | tee "outdir/cele-k${1}.out"

# shakespeare params   
#python3 mlhead.py -dataset shakespeare -model stacked_lstm -t large -lr 0.9 \
#  --clients-per-round 2 --num_clusters "${1}" --num-epochs 1 --num-rounds 40 \
#| tee "outdir/shakespeare-k${1}.out"

# feddist params
#python3 mlhead.py -dataset femnist -model cnn --num-rounds 60  \
# -lr 0.003 --num_clusters -1 --clients-per-round 20 --metric-file femnist_fedist \
# --weight_mode no_size --eval-every 2

# We randomly split the data on each local device into an 80% training set and a 20%
# testing set. We fix the number of selected devices per round to be 10 for all experiments on dataset EMNIST, 
# SHAKESPEARE and 20 for all experiments on CELEBA. We
# also do a grid search on the learning rate based on FedAvg. We do not decay the learning rate through all
# rounds. For FEMNIST, Shakespeare,
# and Celeba, we use the learning rates of 0.003, 0.8, and 0.03. We use a batch size of 10 for all
# experiments.


# Host and experiments assignment

Delta running No Fed (basically remove all communication of server)

Phoenix 3 running FEDsgd on femnist and FedProx on celeba



In [33]:
!cat femnist-prox-saved.tsv

f0027_29	55	55
f0027_29	39	39
f0027_29	47	47
f0027_29	34	34
f0027_29	47	47
f0027_29	55	55
f0027_29	55	55
f0027_29	47	47
f0027_29	59	59
f0027_29	15	15
f0027_29	49	49
f0027_29	13	13
f0027_29	47	47
f0027_29	29	29
f0027_29	37	37
f0027_29	54	54
f0027_29	24	24
f0027_29	18	18
f0027_29	43	43
f0027_29	40	40
f0027_29	15	15
f0027_29	40	40
f0027_29	40	40
f0027_29	44	44
f0027_29	27	27
f0027_29	40	40
f0027_29	39	39
f0027_29	24	24
f0027_29	10	10
f0027_29	30	30
f0027_29	24	24
f0067_25	15	15
f0067_25	27	27
f0067_25	18	18
f0067_25	21	21
f0067_25	27	27
f0067_25	19	19
f0067_25	23	23
f0067_25	30	30
f0067_25	28	28
f0067_25	13	13
f0067_25	30	30
f0067_25	28	28
f0067_25	29	29
f0067_25	43	43
f0067_25	24	24
f0067_25	15	15
f0067_25	24	24
f0067_25	25	25
f0067_25	13	13
f0067_25	25	25
f0067_25	18	18
f0067_25	4	4
f0067_25	1	1
f0067_25	1	1
f0067_25	8	8
f0067_25	6	6
f0086_47	19	19
f0086_47	14	14
f0086_47	9	9
f0086_47	4	4
f0086_47	5	5
f0086_47	1	1
f0086_47	0	0
f0086_47	5	5
f0086_47	6	6
f0086_47	8	8
f0086_47	8	8
f0086_47

f0459_12	1	1
f0459_12	8	8
f0459_12	33	33
f0459_12	57	57
f0492_08	28	28
f0492_08	40	40
f0532_09	54	54
f0532_09	58	58
f0532_09	59	59
f0532_09	47	47
f0532_09	45	45
f0532_09	1	1
f0532_09	1	1
f0532_09	8	8
f0532_09	1	1
f0532_09	9	9
f0532_09	2	2
f0532_09	9	9
f0532_09	4	4
f0532_09	8	8
f0532_09	4	4
f0532_09	1	1
f0532_09	4	4
f0532_09	2	2
f0532_09	23	23
f0532_09	36	36
f0532_09	18	18
f0532_09	18	18
f0532_09	29	29
f0532_09	18	18
f0532_09	25	25
f0532_09	30	30
f0532_09	16	16
f0567_25	45	45
f0567_25	14	14
f0567_25	26	26
f0567_25	24	24
f0567_25	35	35
f0567_25	24	24
f0567_25	55	55
f0567_25	13	13
f0567_25	40	40
f0567_25	36	36
f0567_25	47	47
f0567_25	10	10
f0567_25	40	40
f0567_25	40	40
f0567_25	22	22
f0567_25	18	18
f0567_25	24	24
f0567_25	29	29
f0567_25	53	53
f0567_25	37	37
f0567_25	55	55
f0567_25	22	22
f0567_25	49	49
f0567_25	18	18
f0567_25	25	25
f0567_25	9	9
f0567_25	6	6
f0567_25	3	3
f0567_25	4	4
f0567_25	1	1
f0567_25	3	3
f0567_25	0	0
f0567_25	8	8
f0587_34	2	2
f0587_34	23	23
f0587_34	40	40
f0587_34	12	1